<a href="https://colab.research.google.com/github/zw2497/Twitter_Stream_Processing/blob/master/PySpark_Structured_Streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Java, Spark, and Findspark
This installs Apache Spark 2.4.0, Java 8, and [Findspark](https://github.com/minrk/findspark), a library that makes it easy for Python to find Spark.

In [1]:
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q https://www-us.apache.org/dist/spark/spark-2.4.2/spark-2.4.2-bin-hadoop2.7.tgz
# !tar xf spark-2.4.2-bin-hadoop2.7.tgz
# !pip -q install findspark

#### Set Environment Variables
Set the locations where Spark and Java are installed.

In [ ]:
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-2.4.2-bin-hadoop2.7"

In [ ]:
!pip install -U textblob
!python -m textblob.download_corpora

# Start a SparkSession
This will start a local Spark session.

In [1]:
%%writefile ./app.py
from pyspark.sql.functions import udf, get_json_object, explode, window, concat_ws, count, avg, current_timestamp
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType
from pyspark.sql.functions import pandas_udf, PandasUDFType

from textblob import TextBlob

from sklearn import linear_model

import pandas as pd
import numpy as np

import time

spark = SparkSession \
    .builder \
    .appName("TwitterTopk") \
    .getOrCreate()
spark.sparkContext.setLogLevel('FATAL')

df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "35.243.144.79:9092") \
  .option("subscribe", "tweepyv1") \
  .option("startingOffsets", "latest") \
  .option("failOnDataLoss", "false") \
  .load()

df.createOrReplaceTempView("raw")
df = spark.sql("""select decode(value, 'utf-8') as value, timestamp 
                  from raw""");

@udf(FloatType())
def senti(x):
    blob = TextBlob(x)
    s = []
    for sentence in blob.sentences:
        s.append(sentence.sentiment.polarity)
    return sum(s)/len(s)

df = df.select('timestamp',\
               get_json_object('value', '$.entities.hashtags[0].text').alias("hashtag"), \
               senti(get_json_object('value', '$.text')).alias("sentiment"))
df = df.filter(df.hashtag.isNotNull())

"""## Trend detection"""
df.createOrReplaceTempView("datatrend")
dftrend = spark.sql("""
select distinct hashtag, count(*) as count_num, avg(sentiment) as sentiment, now() as timestamp
from datatrend
group by hashtag, window(timestamp, "180 seconds", "10 seconds")
""")

@pandas_udf("key string, value double", PandasUDFType.GROUPED_MAP)  # doctest: +SKIP
def trend_udf(key, pdf):
    reg = linear_model.LinearRegression()
    reg.fit(np.array(pd.to_datetime(pdf.timestamp).astype('int')).reshape(-1,1), np.array(pdf.count_num).reshape(-1,1))
    return pd.DataFrame([key + (reg.coef_[0][0],)])

dftrend = dftrend.groupby('hashtag').apply(trend_udf)

"""# Top k"""

dfslow = df.withWatermark("timestamp", "1 seconds")\
        .groupby(df.hashtag, window("timestamp", "500 seconds", "60 seconds"))\
        .agg(count('hashtag').alias('countn'), avg('sentiment').alias('sentiment'))

dfslow = dfslow.select(concat_ws(' ', dfslow.hashtag, dfslow.countn, dfslow.sentiment).alias('value'), current_timestamp().alias('key'))


dffast = df.withWatermark("timestamp", "1 seconds")\
        .groupby(df.hashtag, window("timestamp", "180 seconds", "10 seconds"))\
        .agg(count('hashtag').alias('countn'), avg('sentiment').alias('sentiment'))

dffast = dffast.select(concat_ws(' ', dffast.hashtag, dffast.countn, dffast.sentiment).alias('value'), current_timestamp().alias('key'))

query1 = dfslow \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("kafka") \
  .outputMode("update") \
  .option("kafka.bootstrap.servers", "35.243.144.79:9092") \
  .option("topic", "slow") \
  .option("checkpointLocation", "./logslow") \
  .trigger(processingTime='60 seconds') \
  .start()

query2 = dffast \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("kafka") \
  .outputMode("update") \
  .option("kafka.bootstrap.servers", "35.243.144.79:9092") \
  .option("topic", "fast") \
  .option("checkpointLocation", "./logfast") \
  .trigger(processingTime='10 seconds') \
  .start()


query = dftrend \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("kafka") \
  .outputMode("complete") \
  .option("kafka.bootstrap.servers", "35.243.144.79:9092") \
  .option("topic", "trend") \
  .option("checkpointLocation", "./logtrend") \
  .trigger(processingTime='10 seconds') \
  .start()

query2.awaitTermination()
query1.awaitTermination()
query.awaitTermination()

Overwriting ./app.py


In [ ]:
!./spark-2.4.2-bin-hadoop2.7/bin/spark-submit --packages org.apache.spark:spark-sql-kafka-0-10_2.12:2.4.2 app.py

Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/spark-2.4.2-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ac520da2-e4f0-47e3-a9df-6844a11171a8;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;2.4.2 in central
	found org.apache.kafka#kafka-clients;2.0.0 in central
	found org.lz4#lz4-java;1.4.0 in central
	found org.xerial.snappy#snappy-java;1.1.7.3 in central
	found org.slf4j#slf4j-api;1.7.16 in central
	found org.spark-project.spark#unused;1.0.0 in central
:: resolution report :: resolve 436ms :: artifacts dl 10ms
	:: modules in use:
	org.apache.kafka#kafka-clients;2.0.0 from central in [default]
	org.apache.spark#spark-sql-kafka-0-10_2.12;2.4.2 from central in [default]
	org.lz4#lz4-java;1.